In [53]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(palette="pastel")
plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option("display.max_columns", None)

COLUMNAS_GRD = [
    "RUT",
    "SEXO",
    # "EDAD_CANT",
    "PREVI",
    "BENEF",
    "NOM_HOPI_P",
    "FECHA_INGRESO",
    "FECHA_EGRESO",
    "ANO_EGR",
    "DIAS_ESTAD",
    "DIAG1",
    "DIAG2",
    "DIAG3",
    "INTERV_Q",
    "COD_FONASA",
    "INTERV_Q_PPAL",
    "INTERV_Q_2",
    "INTERV_Q_3",
    "T_DIAG1",
    "T_DIAG2",
    "T_DIAG3",
    "PROCED",
    "PROCED_PPAL",
    "PROCED_2",
    "PROCED_3",
]

COLUMNA_GRD_INTERNO = [
    "rut",
    "sexo_(cod)",
    "prevision_(cod)",
    "hospital_de_procedencia_(cod)",
    "fecha_ingreso",
    "fecha_de_egreso",
    "ano_de_egreso",
    "estancias_totales",
    "diag_01_principal_(cod)",
    "diag_01_principal_(des)",
    "diag_02_secundario_(cod)",
    "diag_02_secundario_(des)",
    "diag_03_secundario_(cod)",
    "diag_03_secundario_(des)",
    "proced_01_principal_(cod)",
    "proced_01_principal_(des)",
]

In [55]:
# Lee GRD del INT
df_sabanas = pd.read_csv("../data/processed/grd_sabanas.csv", usecols=COLUMNAS_GRD)
# Formatea fechas de GRD
df_sabanas["FECHA_INGRESO"] = pd.to_datetime(df_sabanas["FECHA_INGRESO"])
df_sabanas["FECHA_EGRESO"] = pd.to_datetime(df_sabanas["FECHA_EGRESO"])

# Lee GRD Interno del INT
df_grd_interno = pd.read_csv("../data/processed/grd_interno.csv", usecols=COLUMNA_GRD_INTERNO)
# Formatea fechas de GRD
df_grd_interno["fecha_ingreso"] = pd.to_datetime(df_grd_interno["fecha_ingreso"])
df_grd_interno["fecha_de_egreso"] = pd.to_datetime(df_grd_interno["fecha_de_egreso"])

# Lee la base de pabellon del INT
df_pabellon = pd.read_csv(
    "../data/external/datos_limpios_pabellon.csv", encoding="latin-1", sep=";"
)
# Formatea fechas de Pabellon
df_pabellon["fecha"] = pd.to_datetime(df_pabellon["fecha"])

# Formatea duraciones de operaciones
df_pabellon["duracion"] = pd.to_timedelta(df_pabellon["duracion"])

/var/folders/wy/2slj_t7s2vz19v2h2926qb340000gn/T/ipykernel_87913/2510431287.py:2: DtypeWarning: Columns (80,156) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sabanas = pd.read_csv("../data/processed/grd_sabanas.csv", usecols=COLUMNAS_GRD)


In [56]:
# Limpia la base de pabellon, sacando las suspendidas y sin duracion
df_pabellon_limpio = df_pabellon.copy()
df_pabellon_limpio = df_pabellon_limpio.query("suspendida == 'NO' and duracion.notna()")

In [57]:
# Une la base de GRD y Pabellon segun el RUT del Paciente
grd_y_pabellon = df_sabanas.merge(
    df_pabellon_limpio, how="inner", left_on="RUT", right_on="ID_PACIENTE"
)

# Filtra la union de GRD y Pabellon para dejar las Int. Q que se hicieron dentro del egreso
grd_y_pabellon = grd_y_pabellon.query("fecha >= FECHA_INGRESO and fecha <= FECHA_EGRESO")

In [58]:
resumen_duraciones = (
    grd_y_pabellon.groupby(["ANO_EGR", "DIAG1"])["duracion"].describe().reset_index()
)

In [65]:
COLUMNAS_A_MOSTRAR = [
    # "rut",
    "fecha_ingreso",
    "fecha_de_egreso",
    "ano_de_egreso",
    "estancias_totales",
    "diag_01_principal_(cod)",
    "diag_01_principal_(des)",
    "proced_01_principal_(cod)",
    "proced_01_principal_(des)",
    "fecha",
    "primer_diagnostico",
    "nombre_de_la_operacion",
    "duracion",
]

# Une la base de GRD Interno y Pabellon segun el RUT del paciente
grd_interno_y_pabellon = df_grd_interno.merge(
    df_pabellon_limpio, how="inner", left_on="rut", right_on="ID_PACIENTE"
)

# Filtra la union de GRD y Pabellon para dejar las Int. Q que se hicieron dentro del egreso
grd_interno_y_pabellon = grd_interno_y_pabellon.query(
    "fecha >= fecha_ingreso and fecha <= fecha_de_egreso"
)

In [82]:
AGRUPACION_A_OBTENER = [
    "ano_de_egreso",
    "diag_01_principal_(cod)",
    "diag_01_principal_(des)",
    "proced_01_principal_(cod)",
    "proced_01_principal_(des)",
]
# Obtiene resumen de duraciones por anio, diagnostico y tipo de operacion
resumen_grd_interno_duraciones_por_operacion = (
    grd_interno_y_pabellon.groupby(AGRUPACION_A_OBTENER)["duracion"].describe().reset_index()
)

# Ordena los resultados por la cantidad de int. q. realizadas para cada diag
resumen_grd_interno_duraciones_por_operacion = (
    resumen_grd_interno_duraciones_por_operacion.sort_values(
        ["ano_de_egreso", "diag_01_principal_(cod)", "diag_01_principal_(des)", "count"],
        ascending=False,
    )
)

In [84]:
resumen_grd_interno_duraciones_por_operacion.set_index(AGRUPACION_A_OBTENER)

count  \
ano_de_egreso diag_01_principal_(cod) diag_01_principal_(des)                            proced_01_principal_(cod) proced_01_principal_(des)                                  
2022          Z47.0                   Cuidados posteriores a la extracción de placa u... 78.61                     EXTRACCION DISPOSITIVO IMPLANTADO EN HUESO.CLAV...     9   
                                                                                         78.91                     INSERCION ESTIMULADOR CRECIMIENTO HUESO.ESCAPUL...     1   
              Z45.0                   Asistencia y ajuste de marcapaso cardíaco          37.87                     SUSTITUCION MARCAPASOS CAMARA DOBLE                   17   
                                                                                         37.85                     SUSTITUCION MARCAPASOS CAMARA UNICA NEOM               6   
                                                                                         0.50                      IMPLANTACION MARCAPASOS RESINCRONIZACION CARDIA...     1   
...                                                                                                                                                                     ...   
2015          A41.8                   Otras septicemias especificadas                    39.65                     OXIGENACION EXTRACORPOREA MEMBRANA                     3   
              A15.6                   Pleuresía tuberculosa, confirmada bacteriológic... 34.02                     TORACOTOMIA EXPLORATORIA                               1   
                                                                                         34.21                     TORACOSCOPIA TRANSPLEURAL                              1   
                                                                                         34.24                     BIOPSIA PLEURA                                         1   
              A15.2                   Tuberculosis del pulmón, confirmada histológica... 34.22                     MEDIASTINOSCOPIA                                       1   

                                                                                                                                                                                            mean  \
ano_de_egreso diag_01_principal_(cod) diag_01_principal_(des)                            proced_01_principal_(cod) proced_01_principal_(des)                                                       
2022          Z47.0                   Cuidados posteriores a la extracción de placa u... 78.61                     EXTRACCION DISPOSITIVO IMPLANTADO EN HUESO.CLAV...            0 days 02:23:20   
                                                                                         78.91                     INSERCION ESTIMULADOR CRECIMIENTO HUESO.ESCAPUL...            0 days 01:45:00   
              Z45.0                   Asistencia y ajuste de marcapaso cardíaco          37.87                     SUSTITUCION MARCAPASOS CAMARA DOBLE                 0 days 01:21:10.588235294   
                                                                                         37.85                     SUSTITUCION MARCAPASOS CAMARA UNICA NEOM                      0 days 01:10:50   
                                                                                         0.50                      IMPLANTACION MARCAPASOS RESINCRONIZACION CARDIA...            0 days 01:45:00   
...                                                                                                                                                                                          ...   
2015          A41.8                   Otras septicemias especificadas                    39.65                     OXIGENACION EXTRACORPOREA MEMBRANA                            0 days 01:31:40   
              A15.6                   Pleuresía tuberculosa, confirmada bacteriológic... 34.02                     TORACOTOMIA